In [46]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from elopy.elo import Elo
import datetime

# Read recipe inputs
# Dataset nfl_scores renamed to nfl_scores_post_Div_Rd by jjorgenson on 2023-02-02 03:34:46
spreadspoke_scores = dataiku.Dataset("nfl_scores_post_Div_Rd")
games_df = spreadspoke_scores.get_dataframe()

In [47]:
games_df.head()

schedule_date  schedule_season schedule_week  schedule_playoff           team_home  score_home  score_away        team_away team_favorite_id  spread_favorite  over_under_line         stadium  stadium_neutral  weather_temperature  weather_wind_mph  weather_humidity weather_detail
0 1966-09-02 00:00:00+00:00             1966             1             False      Miami Dolphins        14.0        23.0  Oakland Raiders              NaN              NaN              NaN     Orange Bowl            False                 83.0               6.0              71.0            NaN
1 1966-09-03 00:00:00+00:00             1966             1             False      Houston Oilers        45.0         7.0   Denver Broncos              NaN              NaN              NaN    Rice Stadium            False                 81.0               7.0              70.0            NaN
2 1966-09-04 00:00:00+00:00             1966             1             False  San Diego Chargers        27.0         7.0    Buffalo Bills              NaN              NaN              NaN  Balboa Stadium            False                 70.0               7.0              82.0            NaN
3 1966-09-09 00:00:00+00:00             1966             2             False      Miami Dolphins        14.0        19.0    New York Jets              NaN              NaN              NaN     Orange Bowl            False                 82.0              11.0              78.0            NaN
4 1966-09-10 00:00:00+00:00             1966             1             False   Green Bay Packers        24.0         3.0  Baltimore Colts              NaN              NaN              NaN   Lambeau Field            False                 64.0               8.0              62.0            NaN

In [115]:
days_back = int(dataiku.get_custom_variables()["days_back"])

start = datetime.datetime.now() - datetime.timedelta(days=days_back)
qry = f'schedule_date>\'{start}+00:00\' & score_home.notna()'
hist_games_df = games_df.query(qry, engine='python')

In [116]:
len(hist_games_df)

285

In [117]:
teams = {}
for team in hist_games_df['team_home'].unique():
    teams[team] = Elo(start_elo=1600, k=20, hca = 100) 
    
for idx, row in hist_games_df.sort_values(['schedule_date']).iterrows():
    if row['score_home'] > row['score_away']:
        winby = row['score_home'] - row['score_away']
        teams[row['team_home']].play_game(teams[row['team_away']], winby, is_home=True)
    else:
        winby = row['score_away'] - row['score_home']
        teams[row['team_away']].play_game(teams[row['team_home']], winby, is_home=False)

#     sprd = 0-teams[row['team_home']].point_spread(teams[row['team_away']], is_home=True)

    hist_games_df.at[idx, 'home_elo'] = teams[row['team_home']].elo
    hist_games_df.at[idx, 'away_elo'] = teams[row['team_away']].elo
#     hist_games_df.at[idx, 'pred_spread'] = sprd

## Record the team ELO Ratings

In [118]:
results = []
for team in hist_games_df['team_home'].unique():
    print(team, teams[team].elo)
    results.append({'team': team, 'rating': teams[team].elo})

Tennessee Titans 1601.6545002187388
Arizona Cardinals 1532.5063118132275
Green Bay Packers 1585.943317078561
Atlanta Falcons 1566.3710007375169
Carolina Panthers 1552.4293614363144
Cincinnati Bengals 1673.0380769119447
Dallas Cowboys 1676.5560922292768
Houston Texans 1521.1768792992887
Kansas City Chiefs 1684.2959021995196
Las Vegas Raiders 1601.5901284410527
Minnesota Vikings 1611.0526812355545
New England Patriots 1602.4724604350724
New York Jets 1599.857154029847
Philadelphia Eagles 1677.508653533402
Seattle Seahawks 1608.1657425422964
New Orleans Saints 1573.90182065471
Baltimore Ravens 1616.7646734101909
Buffalo Bills 1698.4696075373583
Chicago Bears 1555.356441332251
Indianapolis Colts 1547.8899392788824
Los Angeles Chargers 1592.2488321029002
San Francisco 49ers 1683.981981014954
Washington Football Team 1589.1489967449327
Pittsburgh Steelers 1577.6497499750224
Denver Broncos 1549.1864615704826
Cleveland Browns 1590.407203522086
Detroit Lions 1602.2557367929157
Jacksonville Jagu

## Grab the upcoming games

In [95]:
days_ahead = int(dataiku.get_custom_variables()["days_ahead"])

start = datetime.datetime.now() + datetime.timedelta(days=days_ahead)
qry = f'schedule_date<\'{start}+00:00\' & not score_home.notna()'
upcoming_games_df = games_df.query(qry, engine='python')

In [119]:
# upcoming_games_df.head()

## Computing the odds of a home team win

In [122]:
tw_pred = []
for idx, row in upcoming_games_df.iterrows():
    tw_pred.append({'home_team':row['team_home'],
                    'away_team':row['team_away'],
                    'date': row['schedule_date'].strftime("%Y-%m-%d"),
                    'spread': 0-(teams[row['team_home']].point_spread(teams[row['team_away']], is_home=True)),
                    'home_odds':teams[row['team_home']].win_probs(teams[row['team_away']], is_home=True)})

In [123]:
tw_pred

[{'home_team': 'New York Jets',
  'away_team': 'Jacksonville Jaguars',
  'date': '2022-12-22',
  'spread': -3.8087001696249883,
  'home_odds': 0.6488276376867776},
 {'home_team': 'Baltimore Ravens',
  'away_team': 'Atlanta Falcons',
  'date': '2022-12-24',
  'spread': -5.371202595452644,
  'home_odds': 0.7038575851367599},
 {'home_team': 'Carolina Panthers',
  'away_team': 'Detroit Lions',
  'date': '2022-12-24',
  'spread': -1.7919151658356651,
  'home_odds': 0.5717078616157605},
 {'home_team': 'Chicago Bears',
  'away_team': 'Buffalo Bills',
  'date': '2022-12-24',
  'spread': 1.539755935896689,
  'home_odds': 0.4382716775485483},
 {'home_team': 'Cleveland Browns',
  'away_team': 'New Orleans Saints',
  'date': '2022-12-24',
  'spread': -4.160906530977717,
  'home_odds': 0.6616506810254847},
 {'home_team': 'Dallas Cowboys',
  'away_team': 'Philadelphia Eagles',
  'date': '2022-12-24',
  'spread': -3.5374085248526757,
  'home_odds': 0.6388007602245453},
 {'home_team': 'Kansas City Chi

In [124]:
# Write ELO outputs
# Dataset elo_results renamed to elo_results_post_Div_Rd by jjorgenson on 2023-02-02 03:34:56
elo_results = dataiku.Dataset("elo_results_post_Div_Rd")
elo_results.write_with_schema(pd.DataFrame.from_dict(results))

33 rows successfully written (2jE0pqE4Tm)


In [125]:
# Dataset upcoming_game_probabilities renamed to upcoming_game_probabilities_post_Div_Rd by jjorgenson on 2023-02-02 03:35:13
upcoming_results = dataiku.Dataset("upcoming_game_probabilities_post_Div_Rd")
upcoming_results.write_with_schema(pd.DataFrame.from_dict(tw_pred))

17 rows successfully written (QjH2wZgO9P)
